# **Structured Output**

Structured output allows agents to return data in a specific, predictable format. Instead of parsing natural language responses, you get structured data in the form of JSON objects, Pydantic models, or dataclasses that your application can directly use. 

LangChain’s **create_agent** handles structured output automatically. The user sets their desired structured output schema, and when the model generates the structured data, it’s captured, validated, and **returned in the `'structured_response'` key** of the agent’s state.

In [1]:
# Setup API Key

f = open('keys/.gemini.txt')

GOOGLE_API_KEY = f.read()

In [2]:
# Setup API Key

f = open('keys/.openai_api_key.txt')

OPENAI_API_KEY = f.read()

In [3]:
# Setup API Key

f = open('keys/.groq_api_key.txt')

GROQ_API_KEY = f.read()

In [16]:
# Import ChatModel
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq

# Pass the standard parameters during initialization
google_chat_model = ChatGoogleGenerativeAI(api_key=GOOGLE_API_KEY, 
                                           model="gemini-2.5-flash", 
                                           temperature=1)

openai_chat_model = ChatOpenAI(api_key=OPENAI_API_KEY, 
                               model="gpt-4o-mini", 
                               temperature=1)

groq_chat_model = ChatGroq(api_key=GROQ_API_KEY, 
                           model="openai/gpt-oss-20b", 
                           temperature=1)

In [5]:
from pydantic import BaseModel, Field

class ContactInfo(BaseModel):
    """Contact information for a person."""
    name: str = Field(description="The name of the person")
    email: str = Field(description="The email address of the person")
    phone: str = Field(description="The phone number of the person")

## **Response Format**

**response_format** parameter inside **create_agent()** controls how the agent returns structured data. To apply structured output, you should either use ProviderStrategy (default) or ToolStrategy.

- **ProviderStrategy**: Some model providers support structured output natively through their APIs (e.g. OpenAI, Grok, Gemini). This is the most reliable method when available.
- **ToolStrategy**: For models that don’t support native structured output, LangChain uses tool calling to achieve the same result. This works with all models that support tool calling, which is most modern models.


#### **Import**
```python
from langchain.agents.structured_output import ProviderStrategy, ToolStrategy
```

In [13]:
from langchain.agents import create_agent
from langchain.agents.structured_output import ProviderStrategy, ToolStrategy

agent = create_agent(
    model=openai_chat_model,
    response_format=ContactInfo,  # Auto-selects ProviderStrategy
    system_prompt="""You are a helpful assistant. 
                     Be concise and accurate. 
                     You are suppose to extract contact info from the user inputs.
                     """
)

In [10]:
result = agent.invoke({
    "messages": [{"role": "human", "content": "My name is John Doe, email john@example.com, and number (555) 123-4567"}]
})

In [12]:
print(result["structured_response"])

name='John Doe' email='john@example.com' phone='(555) 123-4567'


## **Applying with Groq with ToolStrategy**

```python
class ToolStrategy(schema):
    schema: schema,
    tool_message_content: str | None,
    handle_errors: ERROR_HANDLING_LOGIC
```

- **tool_message_content:** The tool_message_content parameter allows you to customize the message that appears in the conversation history when structured output is generated as per the given schema.
- **handle_errors:** Error handling strategy for structured output validation failures. By default ERROR_HANDLING_LOGIC is set to boolean value True.

In [31]:
from typing import Union
from langchain.agents import create_agent
from langchain.agents.structured_output import ProviderStrategy, ToolStrategy

agent = create_agent(
    model=groq_chat_model,
    response_format=ToolStrategy(schema=Union[ContactInfo,]),  # Default: handle_errors=True
    system_prompt="""You are a helpful assistant. 
                     Be concise and accurate. 
                     You are suppose to extract contact info from the user inputs.
                     """
)

In [32]:
result = agent.invoke({
    "messages": [{"role": "human", "content": "My name is John Doe, email john@example.com, and number (555) 123-4567"}]
})

In [33]:
print(result["structured_response"])

name='John Doe' email='john@example.com' phone='(555) 123-4567'


## **Next**

Use the following reference for the Custom Error Handling.

Reference: https://docs.langchain.com/oss/python/langchain/structured-output#param-schema-1